In [1]:
from nltk import pos_tag
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk 
from itertools import chain
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\giorg\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r'C:\Users\giorg\clean_text.csv', encoding = 'utf-8-sig')

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LsiModel
import spacy

# Pre-Processing

In [4]:
text = df['text']
text = list(text)


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

text_words = list(sent_to_words(text))

print(text_words[:1])

[['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [6]:
bigram = gensim.models.Phrases(text_words, min_count=1, threshold=10) 
trigram = gensim.models.Phrases(bigram[text_words], threshold=10)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[text_words[0]]])

['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']


In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Form Bigrams
text_words_bigrams = make_bigrams(text_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
text_lemmatized = lemmatization(text_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(text_lemmatized[:1])

[['folk', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [9]:
id2word = corpora.Dictionary(text_lemmatized)

# Create Corpus
texts = text_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [10]:
id2word[0]

'cytokine'

In [11]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('cytokine', 1),
  ('folk', 1),
  ('paste', 1),
  ('pfizerbiontech', 1),
  ('storm', 1)]]

# Basic LSA model

In [39]:
lsa_model = LsiModel(corpus, id2word=id2word, num_topics=20)

In [40]:
from pprint import pprint

pprint(lsa_model.print_topics())
doc_lda = lsa_model[corpus]

[(0,
  '0.839*"vaccine" + 0.416*"covid" + 0.256*"moderna" + 0.109*"dose" + '
  '0.078*"sputnikv" + 0.075*"pfizer" + 0.074*"covaxin" + '
  '0.071*"pfizerbiontech" + 0.051*"covidvaccine" + 0.037*"india"'),
 (1,
  '-0.909*"moderna" + 0.313*"vaccine" + -0.098*"dose" + -0.095*"covidvaccine" '
  '+ -0.094*"pfizer" + -0.094*"shoot" + 0.085*"sputnikv" + -0.078*"second" + '
  '0.073*"covaxin" + -0.070*"shot"'),
 (2,
  '0.886*"covid" + -0.403*"vaccine" + -0.160*"moderna" + 0.112*"covaxin" + '
  '0.073*"vaccination" + 0.050*"covidvaccine" + -0.039*"sputnikv" + 0.038*"_" '
  '+ 0.021*"pfizerbiontech" + 0.021*"jab"'),
 (3,
  '-0.939*"covaxin" + -0.210*"dose" + 0.152*"covid" + -0.116*"india" + '
  '-0.091*"covishield" + -0.084*"covidvaccine" + 0.057*"vaccine" + '
  '-0.046*"amp" + -0.036*"ocgn" + -0.032*"shoot"'),
 (4,
  '0.929*"dose" + -0.218*"covaxin" + 0.157*"pfizerbiontech" + '
  '0.141*"covidvaccine" + -0.140*"moderna" + -0.059*"vaccine" + -0.057*"shoot" '
  '+ -0.045*"pfizer" + -0.043*"covid" 

In [41]:
# Compute Perplexity
#print('\nPerplexity: ', lsa_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lsa = CoherenceModel(model=lsa_model, texts=text_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)


Coherence Score:  0.2812645704816641


In [27]:
# pprint(lsa_model.top_topics(corpus,topn=10))

# Visualize the results

In [32]:
pyLDAvis.enable_notebook()
lsa_visualization = pyLDAvis.gensim.prepare(lsa_model, corpus, id2word)
pyLDAvis.display(lsa_visualization)

AttributeError: 'LsiModel' object has no attribute 'inference'

# Trying to find ideal number of topics